In [1]:
!pip install transformers pillow torchvision torch


   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.4 MB 7.2 MB/s eta 0:00:02
   ------------ --------------------------- 3.1/10.4 MB 8.4 MB/s eta 0:00:01
   ----------------- ---------------------- 4.5/10.4 MB 7.9 MB/s eta 0:00:01
   -------------------------- ------------- 6.8/10.4 MB 8.4 MB/s eta 0:00:01
   ------------------------------------ --- 9.4/10.4 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------- ----- 2.1/2.4 MB 13.1 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 8.2 MB/s eta 0:00:00


In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

# 1. Cargar el procesador y el modelo TrOCR
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

# 2. Cargar la imagen y convertirla a RGB
img = Image.open("word_dataset/CIRUELA_0.png").convert("RGB")

# 3. Preprocesar la imagen
inputs = processor(images=img, return_tensors="pt")

# 4. Realizar la inferencia (OCR)
outputs = model.generate(**inputs)

# 5. Decodificar el texto generado
text = processor.decode(outputs[0], skip_special_tokens=True)

# 6. Mostrar el texto extraído
print(text)


c:\Users\iagoc\anaconda3\envs\hugging_face\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_att

CIRUELA


In [3]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import csv
import os
import torch
from tqdm import tqdm  # Para barra de progreso opcional

# Cargar el modelo y el processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

IMG_DIR = "word_dataset"
CSV_PATH = "word_dataset/word_labels.csv"

correct = 0
total = 0

with open(CSV_PATH, "r", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Saltar encabezado

    for row in tqdm(reader):
        img_name, label = row
        img_path = os.path.join(IMG_DIR, img_name)

        # Cargar imagen
        img = Image.open(img_path).convert("RGB")
        inputs = processor(images=img, return_tensors="pt")
        outputs = model.generate(**inputs)
        pred_text = processor.decode(outputs[0], skip_special_tokens=True)

        # Evaluación simple
        if pred_text.strip().lower() == label.strip().lower():
            correct += 1
        total += 1

accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}%")


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

Accuracy: 96.40%
